### 1 - Import test data

In [1]:
!pip install yfinance  



In [61]:
!pip install DateTime

In [64]:
from tkinter import *
from tkinter import messagebox
def sending():
    last_message=text_area.get("1.0","end")
    file=open("coin_name.txt","w")
    file.write(last_message)
    file.close
    
    master.destroy()
    return
    


master=Tk()
canvas=Canvas(master,height=450,width=750)
canvas.pack()
frame_top= Frame (master,bg='#add8e6')
frame_top.place(relx=0.1, rely=0.1,relwidth=0.8,relheight=0.1 )
frame_down= Frame (master,bg='#add8e6')
frame_down.place(relx=0.1, rely=0.25,relwidth=0.8,relheight=0.6) 
coin=Label(frame_top, bg='#add8e6', text= "Stock and Coin",font="Verdana 12 bold")
coin.pack()
coin_area=Label(frame_down, text="Enter Coins or Stock Tickers",bg='#add8e6',font="Verdana 12 bold")
coin_area.pack()
text_area=Text(frame_down,height=9,width=50)
text_area.tag_configure('style', foreground= '#bfbfbf', font=('Verdana',7,'bold'))
text_area.pack()
send_button=Button(frame_down, text="Send",command=sending)
send_button.pack(anchor=S)
master.mainloop()

In [65]:
import yfinance as yf
import pandas as pd
import datetime
from datetime import date
coin_read=open("coin_name.txt","r")

today=date.today()
day=today-datetime.timedelta(days=59)
dataF = yf.download(coin_read.read(), start=day, end=today, interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-05-29 00:00:00,1.072271,1.072731,1.072156,1.072616,1.072616,0
2023-05-29 00:15:00,1.072501,1.072961,1.072501,1.072616,1.072616,0
2023-05-29 00:30:00,1.072501,1.072846,1.072501,1.072846,1.072846,0
2023-05-29 00:45:00,1.072501,1.072961,1.072501,1.072501,1.072501,0
2023-05-29 01:00:00,1.072386,1.072731,1.072156,1.072271,1.072271,0
...,...,...,...,...,...,...
2023-07-25 22:45:00,1.105950,1.106072,1.105828,1.106072,1.106072,0
2023-07-25 23:00:00,1.105950,1.105950,1.105461,1.105461,1.105461,0
2023-07-25 23:15:00,1.105461,1.105583,1.105217,1.105583,1.105583,0


### 2 - Define your signal function

In [66]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [67]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

0    3685
2     152
1     146
Name: signal, dtype: int64

### 3 - Connect to the market and execute trades

In [6]:
!pip install APScheduler

In [7]:
!pip install oandapyV20

In [8]:
!pip install oanda-candles

In [68]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [10]:
!pip install config

In [69]:

def get_candles(n):
    access_token='d665cf217afa2678b7190a34ad02a367-8b55629d4aceb33566e6ae8a02f15c40'
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [32]:
!pip install secure-smtplib

In [65]:
!pip install tk

In [70]:
from tkinter import *
from tkinter import messagebox
def trading_job():
    
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    master=Tk()
    canvas=Canvas(master,height=450,width=750)
    canvas.pack()
    frame_top= Frame (master,bg='#add8e6')
    frame_top.place(relx=0.1, rely=0.1,relwidth=0.8,relheight=0.1 )
    frame_down= Frame (master,bg='#add8e6')
    frame_down.place(relx=0.1, rely=0.25,relwidth=0.8,relheight=0.6) 
    coin=Label(frame_top, bg='#add8e6', text= "Stock and Coin",font="Verdana 12 bold")
    coin.pack()
    coin_area=Label(frame_down, text="Our predictions",bg='#add8e6',font="Verdana 12 bold")
    coin_area.pack()
    text_area=Text(frame_down,height=9,width=50)
    text_area.tag_configure('style', foreground= '#bfbfbf', font=('Verdana',7,'bold'))
    text_area.pack()
    showing_text='Take profit Buy={}  \nStop Lose buy={}  \nTake Profit Sell={}  \nStop Lose Sell={}'.format(TPBuy,SLBuy,TPSell,SLSell)
    text_area.insert(END,showing_text,'style')    
    master.mainloop()
    

### 4 - Executing orders automatically with a scheduler

In [71]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()